### Let's import all the useful libraries in the world. So we don't get stuck with an ugly error later on!

In [2]:
import pandas as pd
from geopy.distance import vincenty
import geohash
import numpy as np
import csv
import operator
import collections
import datetime
from datetime import datetime
from collections import defaultdict

### With that out of the way, the next step is to read data from the masterfile! 
(Which has latitude, longitude, PersonID, altitude, elevation -- all the data that we have collected).

We add a column 'ts' with the date-time of the row.

In [3]:
df = pd.read_csv('masterfile.csv')
df['ts'] = pd.to_datetime(df.time)

### Some pre-processing to do: 
* Sort all the ID's in place. so all the rows with the same ID's occur together. GroupBy would also work?
* How many days do we have data for Person x? [Create Dictionary of Persons mapped with Number of Days of Data collected]
* Round off all the timestamps to 5min intervals

In [4]:
df.sort(columns=['ID'], inplace=True)
names=df['ID'].unique().tolist()

people = {}
for n in names:
    select = df.loc[df['ID']==n]
    people[n] = len(select['Day'].unique())

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [12]:
def round_to_5min(t):
    delta = pd.datetools.timedelta(minutes=t.minute%5, 
                               seconds=t.second, 
                               microseconds=t.microsecond)
    t -= delta
    if delta >= pd.datetools.timedelta(minutes=2.5):
        t += pd.datetools.timedelta(minutes=5)
    return t

In [13]:
df['ts10'] = df['ts'].apply(lambda x: round_to_5min(x))

### Now let's delete all those Persons who have less than 5 days of data. 

We don't need you!

In [5]:
drop_ids = [p for p in people if people[p] <= 5]
df = df.loc[~(df['ID'].isin(drop_ids))]

### Let's begin with the real deal.
* def midpoint(p1,p2): calculates midpoints between p1:(lat,lon) and p2:(lat,lon). outputs p3:(lat,lon).
* def cij(piv,i,j,thrs=30): calculates the distance between person i and j for every datapoint. If the distance is <30m, it is counted as an encounter

In [7]:
def midpoint(p1, p2):
    lat1,lon1 = p1 # JFK
    lat2,lon2 =  p2# SIN

    # Compute path from 1 to 2
    g = Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2);

    # Compute midpoint starting at 1
    h1 = Geodesic.WGS84.Direct(lat1, lon1, g['azi1'], g['s12']/2);
    # Alternatively, compute midpoint starting at 2
    h2 = Geodesic.WGS84.Direct(lat2, lon2, g['azi2'], -g['s12']/2);
    return (h2['lat2'],h2['lon2'])

In [8]:
def cij(piv, i,j, thrs=30):
    '''
    Generate the cij i and j
    '''
    #encounters
    i_lat = piv['latitude'][i]
    i_long = piv['longitude'][i]
    j_lat = piv['latitude'][j]
    j_long = piv['longitude'][j]
    i_point = zip(i_lat, i_long)
    j_point = zip(j_lat, j_long)
    ## I am going to hell for this long comprehension, should the mean accuracy also be taken care of
    ## next iteration.
    dist = [vincenty(i_point[i], j_point[i]).meters  if pd.notnull(i_point[i][0]) and pd.notnull(j_point[i][0]) else 9999 for i in range(0, len(i_point))]
    encounters = [1 if x<=thrs else 0 for x in dist]
    return encounters

In [14]:
trial = df[['ts10','ID','lat','lon','accuracy']].drop_duplicates(subset=['ts10','ID'])

In [15]:
piv = pd.pivot_table(trial, index='ts10', columns='ID' ,values = ['lat','lon','accuracy'])

In [16]:
piv = piv.asfreq('5Min', method=None)

In [19]:
piv.groupby(pd.TimeGrouper('D')).count().tail()

lat                                                 \
ID         1rishabhtrivedi adithyapsv anshumanagrwl anuj7chauhan arch.b80   
ts10                                                                        
2016-04-30             288        181             0          189        4   
2016-05-01             253        285             0          237       16   
2016-05-02               0        208             0          208       22   
2016-05-03               0        277             0          189       26   
2016-05-04               0        205             0          250        2   

                                                                          \
ID         arkg1996 bhagyeshvikani bhardwaj.rish coolsush89 desai.deshna   
ts10                                                                       
2016-04-30        0            288           288        253            0   
2016-05-01        2            288           277        288          163   
2016-05-02        0            224            35        288          288   
2016-05-03        2            288           288        202          288   
2016-05-04        0            277           206        284          277   

             ...         accuracy                                             \
ID           ...    rudra.chandak sacheendra.t sagarparikh31 samriddhisimlai   
ts10         ...                                                               
2016-04-30   ...              285            0             1               0   
2016-05-01   ...              287            0             0               0   
2016-05-02   ...              280            0             0               0   
2016-05-03   ...              280            0             0               0   
2016-05-04   ...              282            0             0               0   

                                                                 \
ID         saumyadoshi shaleen.k.gupta tany.dudett umangjparmar   
ts10                                                              
2016-04-30         282               0         286          231   
2016-05-01         229               0         242          208   
2016-05-02         207               0          48          246   
2016-05-03         275               0         272          232   
2016-05-04         167               0         235          119   

                                     
ID         vaibhav29.07.97 vraj2026  
ts10                                 
2016-04-30             143      240  
2016-05-01             254      146  
2016-05-02             114      149  
2016-05-03               0      210  
2016-05-04               0       33  

[5 rows x 138 columns]

In [24]:
def encounter_data(piv, i,j):
    '''
    Generates a global list of all encounters that have happened in the dataset.
    Information need about each encounter as of now
     - timestamp
     - middle point and consequently the geohash
     - the pair
    
    '''
    #global tot_timestamps,tot_lats,tot_longs,tot_geohash,tot_pairs, tot_encounters
    original_i = i
    original_j = j
    
    #encounters
    i_lat = piv['lat'][i]
    i_long = piv['lon'][i]
    j_lat = piv['lat'][j]
    j_long = piv['lon'][j]
    i_point = zip(i_lat, i_long)
    j_point = zip(j_lat, j_long)
    num_days_data_present_i = [1  if pd.notnull(i_point[i][0]) else 0 for i in range(0, len(i_point))]
    num_days_data_present_j = [1  if pd.notnull(j_point[j][0]) else 0 for j in range(0, len(j_point))]
    min_days = min(sum(num_days_data_present_i),sum(num_days_data_present_j))
    dist = [vincenty(i_point[i], j_point[i]).meters  if pd.notnull(i_point[i][0]) and pd.notnull(j_point[i][0]) else 9999 for i in range(0, min_days)]
    encounters = [1 if x<=30 else 0 for x in dist]    
    avg_days = float(sum(encounters))/float(min_days)

    with open('encounter_master_thrsh_30.csv','ab') as f:
        writer = csv.writer(f)
        writer.writerow([original_i,original_j,avg_days,min_days,sum(encounters)])
    return

In [ ]:
import itertools
count = 0
distance = pd.DataFrame()
temp = ''
l = []
for subset in itertools.combinations(keys, 2):
    i = subset[0]
    j = subset[1]
    if temp != i:
        print i
        temp = i
    encounter_data(piv, i, j)
print "finished"